In [ ]:
!pip install transformers pandas openpyxl


In [ ]:
import pandas as pd

df = pd.read_excel("/content/kote.xlsx", engine="openpyxl")
print("✅ 컬럼명 목록:", df.columns.tolist())
df.head()



✅ 컬럼명 목록: ['Unnamed: 0', '연령', '성별', '상황키워드', '신체질환', '감정_대분류', '감정_소분류', '사람문장1', '시스템문장1', '사람문장2', '시스템문장2', '사람문장3', '시스템문장3']


,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,1,청년,남성,"진로,취업,직장",해당없음,불안,두려운,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...,실수하시다니 정말 미안한 마음이 크겠어요.,내 능력이 부족한 거 같은데 그만 다녀야 될거같아.,능력을 올리려면 어떤 방법이 있을까요?,퇴근 후 여가에 회사 일을 더 열심히 해서 피해가 가지 않도록 해야겠어.,꼭 좋은 결과 있길 바라요.
1,2,청년,남성,"진로,취업,직장",해당없음,불안,두려운,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.,큰 프로젝트를 혼자 하셔서 고민이 많겠네요.,나에게 너무 크게 느껴지는 중요한 프로젝트라 버거운 느낌이 들어.,프로젝트를 잘하시기 위해서 어떤 걸 할 수 있나요?,동료 직원에게 도움을 요청해서 같이 해결해야겠어.,동료 직원에게 도움을 요청하기로 하셨군요.
2,3,청년,남성,"진로,취업,직장",해당없음,불안,두려운,상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워.,직장 상사가 무섭게 생기셔서 친해지는 게 어렵군요.,무섭게 생겼는데도 업무를 보려면 친해져야 할 것 같단말이야.,상사분과 친해질 수 있는 방법은 무엇이 있을까요?,먼저 다가가서 말을 걸어볼게.,직장 상사와 친해지시면 좋겠네요.
3,4,청년,남성,"진로,취업,직장",해당없음,불안,두려운,이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다.,첫 직장이라서 정말 떨리시고 긴장되실 것 같아요.,첫 직장이어서 잘 적응을 할 수 있을지 모르겠어.,잘 적응 하시려면 무엇을 할 수 있을까요?,직장 동료와 상사들이랑 친하게 지내야겠어.,직장에 잘 적응하시길 바라요.
4,5,청년,남성,"진로,취업,직장",해당없음,불안,두려운,직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼.,직장 사람들 관계에 대해서 고민이시군요.,내가 낯가림이 심해서 친해질 수 있을지 모르겠어.,직장 사람들과 친해지려면 무슨 방법이 있을까요?,서로 같은 취미를 공유하고 얘기를 나누다 보면 친해질 수 있을 거 같아.,직장 사람들과 좋은 관계를 가지시길 응원해요.


In [6]:

# ✅ 라이브러리 불러오기
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
# ✅ 정확한 Colab 경로로 수정
xlsx_path = "/content/kote1.xlsx"

# 파일 로딩
df = pd.read_excel(xlsx_path, engine="openpyxl")

# 컬럼명 확인
print("✔️ 컬럼명 목록:", df.columns.tolist())
df.head()


✔️ 컬럼명 목록: ['Unnamed: 0', '감정_대분류', '감정_소분류', '사람문장1']


,Unnamed: 0,감정_대분류,감정_소분류,사람문장1
0,1,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,2,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,3,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,4,분노,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,5,분노,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.


In [7]:
text_col = "사람문장1"

texts = df[text_col].dropna().astype(str).tolist()
texts = texts[:10]

# 모델 로딩
model_name = "searle-j/kote_for_easygoing_people"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# 사용할 디바이스 설정 (GPU 사용 가능하면 GPU, 아니면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) # 모델을 선택된 디바이스로 이동

pipe = TextClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    device=device, # 정의된 device 변수 사용
    return_all_scores=True,
    function_to_apply='sigmoid'
)

# 감정 분석
pipe_outputs = pipe(texts)

# 출력 (score ≥ 0.4만)
for i in range(len(pipe_outputs)):
    print(f"\n▶ 문장: {texts[i]}")
    for output in pipe_outputs[i]:
        if output["score"] > 0.4:
            print(f"  - 감정: {output['label']} / 확률: {output['score']:.2f}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/396k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/788k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(



▶ 문장: 일은 왜 해도 해도 끝이 없을까? 화가 난다.
  - 감정: 불평/불만 / 확률: 0.98
  - 감정: 지긋지긋 / 확률: 0.47
  - 감정: 슬픔 / 확률: 0.58
  - 감정: 화남/분노 / 확률: 0.94
  - 감정: 안타까움/실망 / 확률: 0.89
  - 감정: 의심/불신 / 확률: 0.43
  - 감정: 절망 / 확률: 0.73
  - 감정: 짜증 / 확률: 0.95
  - 감정: 힘듦/지침 / 확률: 0.51
  - 감정: 증오/혐오 / 확률: 0.42
  - 감정: 불안/걱정 / 확률: 0.49

▶ 문장: 이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
  - 감정: 불평/불만 / 확률: 0.83
  - 감정: 슬픔 / 확률: 0.78
  - 감정: 화남/분노 / 확률: 0.77
  - 감정: 안타까움/실망 / 확률: 0.79
  - 감정: 절망 / 확률: 0.43
  - 감정: 짜증 / 확률: 0.78

▶ 문장: 회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스트레스 받아. 
  - 감정: 불평/불만 / 확률: 0.89
  - 감정: 지긋지긋 / 확률: 0.42
  - 감정: 화남/분노 / 확률: 0.74
  - 감정: 짜증 / 확률: 0.95
  - 감정: 증오/혐오 / 확률: 0.43

▶ 문장: 직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 섭섭해.
  - 감정: 불평/불만 / 확률: 0.88
  - 감정: 슬픔 / 확률: 0.65
  - 감정: 화남/분노 / 확률: 0.85
  - 감정: 안타까움/실망 / 확률: 0.69
  - 감정: 짜증 / 확률: 0.88

▶ 문장: 얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.
  - 감정: 불평/불만 / 확률: 0.91
  - 감정: 화남/분노 / 확률: 0.91
  - 감정: 짜증 / 확률: 0.93

▶ 문장: 직장에 다니고 있지만 시간만 버

In [ ]:

# 사용자 입력 문장 감정 분석
while True:
    user_input = input("감정 분석할 문장을 입력하세요 (종료하려면 x): ")
    if user_input.lower() == "x":
        break
    result = pipe(user_input)[0]
    print(f"▶ 입력 문장: {user_input}")
    for r in result:
        if r["score"] > 0.0:
            print(f"  - 감정: {r['label']} / 확률: {r['score']:.2f}")
    print()


감정 분석할 문장을 입력하세요 (종료하려면 x): 아기돼지 삼형제 집이 날아갔어요
▶ 입력 문장: 아기돼지 삼형제 집이 날아갔어요
  - 감정: 불평/불만 / 확률: 0.30
  - 감정: 환영/호의 / 확률: 0.05
  - 감정: 감동/감탄 / 확률: 0.10
  - 감정: 지긋지긋 / 확률: 0.23
  - 감정: 고마움 / 확률: 0.03
  - 감정: 슬픔 / 확률: 0.61
  - 감정: 화남/분노 / 확률: 0.43
  - 감정: 존경 / 확률: 0.02
  - 감정: 기대감 / 확률: 0.09
  - 감정: 우쭐댐/무시함 / 확률: 0.06
  - 감정: 안타까움/실망 / 확률: 0.43
  - 감정: 비장함 / 확률: 0.07
  - 감정: 의심/불신 / 확률: 0.08
  - 감정: 뿌듯함 / 확률: 0.05
  - 감정: 편안/쾌적 / 확률: 0.03
  - 감정: 신기함/관심 / 확률: 0.16
  - 감정: 아껴주는 / 확률: 0.12
  - 감정: 부끄러움 / 확률: 0.22
  - 감정: 공포/무서움 / 확률: 0.29
  - 감정: 절망 / 확률: 0.39
  - 감정: 한심함 / 확률: 0.21
  - 감정: 역겨움/징그러움 / 확률: 0.16
  - 감정: 짜증 / 확률: 0.62
  - 감정: 어이없음 / 확률: 0.57
  - 감정: 없음 / 확률: 0.17
  - 감정: 패배/자기혐오 / 확률: 0.28
  - 감정: 귀찮음 / 확률: 0.13
  - 감정: 힘듦/지침 / 확률: 0.42
  - 감정: 즐거움/신남 / 확률: 0.15
  - 감정: 깨달음 / 확률: 0.13
  - 감정: 죄책감 / 확률: 0.15
  - 감정: 증오/혐오 / 확률: 0.16
  - 감정: 흐뭇함(귀여움/예쁨) / 확률: 0.12
  - 감정: 당황/난처 / 확률: 0.78
  - 감정: 경악 / 확률: 0.42
  - 감정: 부담/안_내킴 / 확률: 0.17
  - 감정: 서러움 / 확률: 0.44
  - 감정: 재미없음 / 확률: 0.0

In [ ]:
# 감정 44개를 11개 대분류로 4개씩 라벨링
label_to_group = {
    # 슬픔
    "슬픔": "슬픔", "절망": "슬픔", "불쌍함/연민": "슬픔", "재미없음": "슬픔",
    # 분노
    "화남/분노": "분노", "짜증": "분노", "불평/불만": "분노", "증오/혐오": "분노",
    # 불안
    "불안/걱정": "불안", "공포/무서움": "불안", "당황/난처": "불안", "귀찮음": "불안",
    # 수치
    "부끄러움": "수치", "죄책감": "수치", "패배/자기혐오": "수치", "한심함": "수치",
    # 기쁨
    "기쁨": "기쁨", "즐거움/신남": "기쁨", "행복": "기쁨", "뿌듯함": "기쁨",
    # 감사
    "고마움": "감사", "환영/호의": "감사", "아껴주는": "감사", "흐뭇함(귀여움/예쁨)": "감사",
    # 감탄
    "감동/감탄": "감탄", "깨달음": "감탄", "비장함": "감탄", "기대감": "감탄",
    # 신뢰
    "안심/신뢰": "신뢰", "존경": "신뢰", "편안/쾌적": "신뢰", "의심/불신": "신뢰",
    # 혐오
    "역겨움/징그러움": "혐오", "우쭐댐/무시함": "혐오", "어이없음": "혐오", "지긋지긋": "혐오",
    # 놀람
    "놀람": "놀람", "경악": "놀람", "신기함/관심": "놀람", "무서움": "놀람",
    # 없음 제외
}

# 사용자 입력 감정 분석
while True:
    user_input = input("감정 분석할 문장을 입력하세요 (종료하려면 x): ")
    if user_input.lower() == "x":
        break

    result = pipe(user_input)[0]

    # 0.4 이상 필터링
    filtered = [r for r in result if r["score"] >= 0.4]

    # 그룹별 개수와 score 합 계산
    group_counts = {}
    group_scores = {}

    for r in filtered:
        group = label_to_group.get(r["label"])
        if group:
            group_counts[group] = group_counts.get(group, 0) + 1
            group_scores[group] = group_scores.get(group, 0.0) + r["score"]

    if not group_counts:
        print(f"▶ 입력 문장: {user_input}")
        print("  - 감정: 없음 (0.4 이상 X)")
        print()
        continue

    # 가장 많이 나온 그룹 찾기
    max_count = max(group_counts.values())
    candidate_groups = [g for g, c in group_counts.items() if c == max_count]

    # 동률일 경우 score 합이 높은 그룹 선택
    if len(candidate_groups) == 1:
        final_group = candidate_groups[0]
    else:
        final_group = max(candidate_groups, key=lambda g: group_scores[g])

    print(f"▶ 입력 문장: {user_input}")
    print(f"  - 예측 대분류 감정: {final_group}")
    print()


감정 분석할 문장을 입력하세요 (종료하려면 exit): 나 오늘 너무 피곤해
▶ 입력 문장: 나 오늘 너무 피곤해
  - 예측 대분류 감정: 분노

감정 분석할 문장을 입력하세요 (종료하려면 exit): 아 짜증나
▶ 입력 문장: 아 짜증나
  - 예측 대분류 감정: 분노

감정 분석할 문장을 입력하세요 (종료하려면 exit): 
▶ 입력 문장: 
  - 예측 대분류 감정: 분노

감정 분석할 문장을 입력하세요 (종료하려면 exit): 사람은 언제 철이 들까
▶ 입력 문장: 사람은 언제 철이 들까
  - 예측 대분류 감정: 분노

감정 분석할 문장을 입력하세요 (종료하려면 exit): 너무 행복해
▶ 입력 문장: 너무 행복해
  - 예측 대분류 감정: 기쁨

감정 분석할 문장을 입력하세요 (종료하려면 exit): 늑대가 바람을 불어서 집이 날아갔어
▶ 입력 문장: 늑대가 바람을 불어서 집이 날아갔어
  - 예측 대분류 감정: 불안

감정 분석할 문장을 입력하세요 (종료하려면 exit): exit


In [ ]:
!pip install --upgrade transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
import torch
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score

# 모델 설정
model_name = "searle-j/kote_for_easygoing_people"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

pipe = TextClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
    return_all_scores=True,
    function_to_apply="sigmoid"
)

# 데이터 로딩
df = pd.read_excel("kote3.xlsx", engine="openpyxl")
texts = df["사람문장1"].fillna("").astype(str).tolist()
true_big_labels = df["감정_44분류"].fillna("").astype(str).tolist()

# 예측
pipe_outputs = pipe(texts[:1500])
pred_top_labels = [max(p, key=lambda x: x["score"])["label"] for p in pipe_outputs]

# 정확도 출력
print(classification_report(true_big_labels[:1500], pred_top_labels, digits=3))
print(f"정확도: {accuracy_score(true_big_labels[:1500], pred_top_labels)*100:.2f}%")

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


              precision    recall  f1-score   support

       감동/감탄      1.000     1.000     1.000        21
          경악      1.000     1.000     1.000         1
         고마움      1.000     1.000     1.000        18
         기대감      1.000     1.000     1.000        23
          기쁨      1.000     1.000     1.000       136
         깨달음      1.000     1.000     1.000        32
          놀람      1.000     1.000     1.000        12
       당황/난처      1.000     1.000     1.000       142
      불쌍함/연민      1.000     1.000     1.000         2
       불안/걱정      1.000     1.000     1.000       206
       불평/불만      1.000     1.000     1.000        48
         비장함      1.000     1.000     1.000         2
          슬픔      1.000     1.000     1.000       286
      신기함/관심      1.000     1.000     1.000         3
        아껴주는      1.000     1.000     1.000         6
       안심/신뢰      1.000     1.000     1.000        66
     안타까움/실망      1.000     1.000     1.000       228
        어이없음      1.000    